In [2]:
pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for fpdf: started
  Running setup.py install for fpdf: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: fpdf is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

# Load your class data
df = pd.read_csv("form.csv")

# Rename for easier access
df = df.rename(columns={
    'Name:-': 'Name',
    'Average  % of BTech till 6th Sem': 'BTech%',
    'Mention programming languages you have expertise in:-': 'Languages',
    'Mention your career(Technical)  domain:-': 'Domain',
    'Mention the projects you have made in list like\nProject1 :- name\nDescription of project 1:- what, how, and why and tech stack\nProject 2 :- name\nDescription:-......so on\n': 'Projects',
    'If you have github account then provide the link :-': 'GitHub',
    'If you have linkedin account then provide the link :-': 'LinkedIn',
    'If you have portfolio then provide the link :-': 'Portfolio',
    'Any industry level certification or achievements, then kindly mention:-': 'Certifications',
    'If you have competitive programming account at any platform then provide the link :-': 'CP'
})

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from fpdf import FPDF

# 1. Prepare folders
os.makedirs("reports", exist_ok=True)

# 2. Clean BTech% column (if needed)
df['BTech%'] = df['BTech%'].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# 3. Utility functions
def is_link_present(link):
    return isinstance(link, str) and link.strip() != ""

def extract_project_count(projects_text):
    return str(projects_text).lower().count('project')

def calculate_scores(row):
    scores = {}
    scores['Academics'] = min(float(row['BTech%']) / 10, 10)
    lang_count = len(str(row['Languages']).split(','))
    scores['Coding Skills'] = min(lang_count * 3, 10)
    proj_count = extract_project_count(str(row['Projects']))
    scores['Projects'] = min(proj_count * 3, 10)
    cert_count = 0 if pd.isna(row['Certifications']) else len(str(row['Certifications']).split(','))
    scores['Certifications'] = min(cert_count * 5, 10)
    presence_score = sum([
        is_link_present(row['GitHub']),
        is_link_present(row['LinkedIn']),
        is_link_present(row['Portfolio'])
    ]) * 3.33
    scores['Online Presence'] = min(presence_score, 10)
    return scores

def skill_gap(scores, row):
    gaps = []
    if not is_link_present(row['CP']):
        gaps.append(['Competitive Programming', 'Low', 'Start with LeetCode or HackerRank'])
        
        
    if scores['Certifications'] < 1:
        gaps.append(['Certifications', 'No', 'Add industry certs from Coursera/Udemy'])
    elif scores['Certifications'] < 2:
        gaps.append(['Certifications', 'Low', 'Add industry certs from Coursera/Udemy'])
    elif scores['Certifications'] < 7:
        gaps.append(['Certifications', 'Medium', 'Add industry certs from Coursera/Udemy'])
        
        
    if scores['Projects'] < 1:
        gaps.append(['Projects', 'No', 'Work on real-world or open-source projects'])
    elif scores['Projects'] < 2:
        gaps.append(['Projects', 'Low', 'Work on real-world or open-source projects'])
    elif scores['Projects'] < 7:
        gaps.append(['Projects', 'Medium', 'Work on real-world or open-source projects'])
        
        
    if scores['Online Presence'] < 1:
        gaps.append(['Online Presence', 'No', 'Complete GitHub/LinkedIn/Portfolio profile'])
    elif scores['Online Presence'] < 2:
        gaps.append(['Online Presence', 'Low', 'Complete GitHub/LinkedIn/Portfolio profile'])
    elif scores['Online Presence'] < 7:
        gaps.append(['Online Presence', 'Medium', 'Complete GitHub/LinkedIn/Portfolio profile'])
        
    return gaps

def save_radar_chart(scores, name):
    categories = list(scores.keys())
    values = list(scores.values())
    values += values[:1]
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles, values, linewidth=2, label=name)
    ax.fill(angles, values, alpha=0.25)
    ax.set_thetagrids(np.degrees(angles[:-1]), categories)
    ax.set_ylim(0, 10)
    ax.set_title(f"{name}'s Skill Radar", y=1.1)
    ax.legend(loc='upper right')
    plt.tight_layout()

    filename = f"{name}_radar.png"
    plt.savefig(filename)
    plt.close()
    return filename

# 4. Generate combined PDF
def generate_pdf_report(name, scores, gap_list, radar_path):
    pdf = FPDF()
    pdf.add_page()

    # Title
    pdf.set_font("Arial", "B", 16)
    pdf.cell(200, 10, f"{name}'s Technical Skill Report", ln=True, align='C')

    # Radar Chart
    pdf.image(radar_path, x=40, y=30, w=130)  # Add radar chart image
    pdf.ln(100)  # Push next content down ~100 units to avoid overlap (adjust as needed)
    
    pdf.set_y(150)

    # Skill Gaps Section
    pdf.set_font("Arial", "B", 12)
    pdf.ln(10)
    pdf.cell(0, 10, "Skill Gap Analysis", ln=True)

    pdf.set_font("Arial", "", 11)
    if not gap_list:
        pdf.cell(0, 10, "No significant skill gaps detected. Great work!", ln=True)
    else:
        # Table headers
        pdf.set_fill_color(200, 220, 255)
        pdf.cell(60, 8, "Area", 1, 0, 'C', True)
        pdf.cell(40, 8, "Score", 1, 0, 'C', True)
        pdf.cell(90, 8, "Recommendation", 1, 1, 'C', True)

        # Table rows
        for gap in gap_list:
            pdf.cell(60, 8, gap[0], 1)
            pdf.cell(40, 8, gap[1], 1)
            pdf.cell(90, 8, gap[2], 1)
            pdf.ln()

    # Save PDF
    pdf.output(f"reports/{name.replace(' ', '_')}_Report.pdf")
    os.remove(radar_path)



# 5. Run for all students
for index, row in df.iterrows():
    name = str(row['Name']).strip().replace('/', '-')
    scores = calculate_scores(row)
    radar_path = save_radar_chart(scores, name)
    gaps = skill_gap(scores, row)
    generate_pdf_report(name, scores, gaps, radar_path)
    print(f"✅ PDF Report generated for: {name}")


✅ PDF Report generated for: Amit Khotele
